<a href="https://colab.research.google.com/github/joshfitts34/Capstone/blob/main/Code/image_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reading in the image and resizing to declared size and preprocessing 
img = cv2.imread(pathImage)
img = cv2.resize(img, (widthImg, heightImg))  # resizing to make a square
imgBlank = np.zeros((heightImg, widthImg, 3), np.uint8) # for testing
imgThreshold = preProcess(img)


imgContours = img.copy() 
imgBigContour = img.copy()

contours, hierarchy = cv2.findContours(imgThreshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Finding external contour to use for board
cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 3) # Drawing the contours on the image

biggest, maxArea = biggestContour(contours) # Finding largest contour area and using as board edges
#print(biggest)

if biggest.size != 0:
    biggest = reorder(biggest)
    #print(biggest)
    cv2.drawContours(imgBigContour, biggest, -1, (255, 0, 0), 30) # Showing where the board is
    pts1 = np.float32(biggest) # 
    pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # Setting points for warping image

    matrix = cv2.getPerspectiveTransform(pts1, pts2) 
    imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
    imgDetectedDigits = imgBlank.copy()
    imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY) # Using grayscale 

    
    imgSolvedDigits = imgBlank.copy()
    boxes = splitBoxes(imgWarpColored)
    print(len(boxes))

    #cv2_imshow(boxes[1])
    numbers = getPredection(boxes, model)
    print(numbers)

    imgDetectedDigits = displayNumbers(imgDetectedDigits, numbers, color=(255, 0, 255))
    numbers = np.asarray(numbers)
    posArray = np.where(numbers > 0, 0, 1) # prints value 1 where needs to find digit
    print(posArray)








else:
    print("No Sudoku Found")